# Importando as bibliotecas

In [115]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sn
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados
#from sklearn.metrics import plot_confusion_matrix

In [116]:
# Dataset

In [117]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

In [118]:
# Funções

In [119]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    resultados_matriz = []

    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        seed = i
        X_train, X_test, y_train, y_test = procData.holdout(0.2, seed)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor K-Vizinhos, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = KNeighborsClassifier()
        grid_knn = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_knn.fit(X_train, y_train)
        
        # Treinando do modelo com os melhores parametros encontrados
        grid_knn.best_params_
        Kvizinhos_best = int(grid_knn.best_params_['n_neighbors']) 
        pesos_best = int(grid_knn.best_params_['p'])
        KNN = KNeighborsClassifier(n_neighbors=Kvizinhos_best, metric='minkowski', p = pesos_best)
        KNN.fit(X_train, y_train)

        #testando o modelo
        y_pred = KNN.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        matriz = metrics.confusion_matrix(y_test, y_pred)
        
        f1_score = 2 * (precision * recall) / (precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)
        resultados_matriz.append(matriz)

        #best_parametros = "K-Vizinhos (n_neighbors): "+ str(Kvizinhos_best)+ " | Peso (p): "+str(pesos_best);
        best_parametros = {'n_neighbors': Kvizinhos_best, 'p': pesos_best};
        resultados_parametros.append(best_parametros)



        if exibir_metricas:
            print("Rodada:",i)
            print("Parâmetros:",best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    resultados_gerais['params'] = resultados_parametros
    resultados_gerais['matriz'] = resultados_matriz
    
    return resultados_gerais

In [120]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== RESULTADOS DAS MÉTRICAS DO MODELO ==============================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))

        print ("")
        print ("=================================== MELHORES RESULTADOS ============================================")

        indice_accuracy = np.array(dict_metricas['accuracy'])
        melhor_accuracy = indice_accuracy.flatten()[np.argmax(indice_accuracy)] #retorna o maior valor
        melhor_params_accuracy = dict_metricas['params'][np.argmax(indice_accuracy)]
        melhor_matriz_accuracy = dict_metricas['matriz'][np.argmax(indice_accuracy)]
        
        indice_precision = np.array(dict_metricas['precision'])
        melhor_precision = indice_precision.flatten()[np.argmax(indice_precision)] #retorna o maior valor
        melhor_params_precision = dict_metricas['params'][np.argmax(indice_precision)]
        melhor_matriz_precision = dict_metricas['matriz'][np.argmax(indice_precision)]
        
        indice_recall = np.array(dict_metricas['recall'])
        melhor_recall = indice_recall.flatten()[np.argmax(indice_recall)] #retorna o maior valor
        melhor_params_recall = dict_metricas['params'][np.argmax(indice_recall)]
        melhor_matriz_recall = dict_metricas['matriz'][np.argmax(indice_recall)]
        
        indice_f1 = np.array(dict_metricas['f1'])
        melhor_f1 = indice_f1.flatten()[np.argmax(indice_f1)] #retorna o maior valor
        melhor_params_f1 = dict_metricas['params'][np.argmax(indice_f1)]        
        melhor_matriz_f1 = dict_metricas['matriz'][np.argmax(indice_f1)]
        
        print ("Accuracy")
        print ("Rodada:", np.argmax(indice_accuracy)," | Parâmetro:", melhor_params_accuracy, " | Resultado:",melhor_accuracy)
        print ("Matriz de Confusão:")
        #print (melhor_matriz_accuracy)
        #plot_confusion_matrix(melhor_matriz_accuracy)
        
        sn.heatmap(melhor_matriz_accuracy, annot=True)
        plt.show()
        
        print ("")
        print ("Precision")
        print ("Rodada:", np.argmax(indice_precision)," | Parâmetro:", melhor_params_precision, " | Resultado:",melhor_precision)
        print ("Matriz de Confusão:")
        #print (melhor_matriz_precision)
        #plot_confusion_matrix(melhor_matriz_precision)
        
        sn.heatmap(melhor_matriz_precision, annot=True)
        plt.show()

        print ("")
        print ("Recall:")
        print ("Rodada:", np.argmax(indice_recall)," | Parâmetro:", melhor_params_recall, " | Resultado:",melhor_recall)
        print ("Matriz de Confusão:")
        #print (melhor_matriz_recall)
        #plot_confusion_matrix(melhor_matriz_recall)
        
        sn.heatmap(melhor_matriz_recall, annot=True)
        plt.show()
        
        print ("F1-Score:")
        print ("Rodada:", np.argmax(indice_f1)," | Parâmetro:", melhor_params_f1, " | Resultado:",melhor_f1)
        print ("Matriz de Confusão:")
        #print (melhor_matriz_f1)        
        #plot_confusion_matrix(melhor_matriz_f1)
        
        sn.heatmap(melhor_matriz_f1, annot=True)
        plt.show()

        print ("===================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [121]:
k_range = np.arange(1,25,1) # K-Vizinhos a serem testados no SearchGridCV 
pesos = [1, 2] # 1:Manhattan | 2:Euclidiana
param_grid = {'n_neighbors': k_range, 'p': pesos}
epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True

# Treinando e obtendo as métricas do modelo

In [ ]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('KNN',dict_metricas)

C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[23  5]
 [ 8 84]]
Rodada: 0
Parâmetros: {'n_neighbors': 1, 'p': 1}
Accuracy: 0.8916666666666667
Precision: 0.9438202247191011
Recall: 0.9130434782608695
F1-Score: 0.9281767955801105


[[20 13]
 [ 5 82]]
Rodada: 1
Parâmetros: {'n_neighbors': 1, 'p': 2}
Accuracy: 0.85
Precision: 0.8631578947368421
Recall: 0.9425287356321839
F1-Score: 0.901098901098901


[[13  9]
 [ 7 91]]
Rodada: 2
Parâmetros: {'n_neighbors': 7, 'p': 1}
Accuracy: 0.8666666666666667
Precision: 0.91
Recall: 0.9285714285714286
F1-Score: 0.9191919191919193




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[25  7]
 [ 2 86]]
Rodada: 3
Parâmetros: {'n_neighbors': 3, 'p': 2}
Accuracy: 0.925
Precision: 0.9247311827956989
Recall: 0.9772727272727273
F1-Score: 0.9502762430939227




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


...

[[28 10]
 [ 5 77]]
Rodada: 24
Parâmetros: {'n_neighbors': 1, 'p': 2}
Accuracy: 0.875
Precision: 0.8850574712643678
Recall: 0.9390243902439024
F1-Score: 0.911242603550296


[[23  6]
 [ 7 84]]
Rodada: 25
Parâmetros: {'n_neighbors': 7, 'p': 1}
Accuracy: 0.8916666666666667
Precision: 0.9333333333333333
Recall: 0.9230769230769231
F1-Score: 0.9281767955801106


[[19  7]
 [ 4 90]]
Rodada: 26
Parâmetros: {'n_neighbors': 11, 'p': 1}
Accuracy: 0.9083333333333333
Precision: 0.9278350515463918
Recall: 0.9574468085106383
F1-Score: 0.9424083769633509




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[21 11]
 [ 5 83]]
Rodada: 27
Parâmetros: {'n_neighbors': 3, 'p': 1}
Accuracy: 0.8666666666666667
Precision: 0.8829787234042553
Recall: 0.9431818181818182
F1-Score: 0.9120879120879122


[[20  5]
 [ 6 89]]
Rodada: 28
Parâmetros: {'n_neighbors': 5, 'p': 1}
Accuracy: 0.9083333333333333
Precision: 0.9468085106382979
Recall: 0.9368421052631579
F1-Score: 0.9417989417989417


[[23 11]
 [ 6 80]]
Rodada: 29
Parâmetros: {'n_neighbors': 11, 'p': 2}
Accuracy: 0.8583333333333333
Precision: 0.8791208791208791
Recall: 0.9302325581395349
F1-Score: 0.9039548022598871


[[17 13]
 [ 1 89]]
Rodada: 30
Parâmetros: {'n_neighbors': 5, 'p': 1}
Accuracy: 0.8833333333333333
Precision: 0.8725490196078431
Recall: 0.9888888888888889
F1-Score: 0.9270833333333334




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[20  8]
 [ 5 87]]
Rodada: 31
Parâmetros: {'n_neighbors': 4, 'p': 2}
Accuracy: 0.8916666666666667
Precision: 0.9157894736842105
Recall: 0.9456521739130435
F1-Score: 0.9304812834224598


[[21  7]
 [ 1 91]]
Rodada: 32
Parâmetros: {'n_neighbors': 9, 'p': 2}
Accuracy: 0.9333333333333333
Precision: 0.9285714285714286
Recall: 0.9891304347826086
F1-Score: 0.9578947368421052


[[28 10]
 [ 1 81]]
Rodada: 33
Parâmetros: {'n_neighbors': 7, 'p': 1}
Accuracy: 0.9083333333333333
Precision: 0.8901098901098901
Recall: 0.9878048780487805
F1-Score: 0.9364161849710982


[[22  8]
 [ 2 88]]
Rodada: 34
Parâmetros: {'n_neighbors': 7, 'p': 1}
Accuracy: 0.9166666666666666
Precision: 0.9166666666666666
Recall: 0.9777777777777777
F1-Score: 0.946236559139785


